In [1]:
## re-run the rejection reason final model after that test
from cashe import get_train_test_split, get_training_inputs, drop_nomodel_columns


from src.utilities_rejection import RejectionReasonLabeling
from src.data_local import MergedDataPreprocessing
from src.model_train import ModelTrainer
import pandas as pd
import numpy as np

df_train, df_test = get_train_test_split('data/HJH/12-06-2024')
df_train = df_train[df_train['OUTCOME'] !='APPROVED']
df_test = df_test[df_test['OUTCOME'] !='APPROVED']

train_rej = RejectionReasonLabeling(df_train)
train_rej.recoginze_label()
train_rej = RejectionReasonLabeling(df_test)
train_rej.recoginze_label()

def analyze_error_code(error_value):

    df_train['NPHIES_LABEL'] = np.where(df_train['NPHIES_CODE'] == error_value, 1, 0)
    df_test['NPHIES_LABEL']  = np.where( df_test['NPHIES_CODE']  == error_value, 1, 0)
    X_train, y_train, X_test, y_test = get_training_inputs(df_train, df_test)
    X_train = drop_nomodel_columns(X_train);  X_test = drop_nomodel_columns(X_test)
    X_train.drop(columns=['NPHIES_CODE'],inplace=True); X_test.drop(columns=['NPHIES_CODE'],inplace=True)

    preprocessing_train = MergedDataPreprocessing(X_train)
    X_train_prep = preprocessing_train.columns_prep()

    preprocessing_test = MergedDataPreprocessing(X_test)
    X_test_prep = preprocessing_test.columns_prep()

    X_train_encoded = preprocessing_train.column_embedding(X_train_prep)
    X_test_encoded = preprocessing_train.column_embedding(X_test_prep)

    X_train_encoded = preprocessing_train.column_embedding(X_train_encoded,is_service=False)
    X_test_encoded = preprocessing_train.column_embedding(X_test_encoded,is_service=False)

    X_train_encoded.fillna(0,inplace=True)
    X_test_encoded.fillna(0,inplace=True)

    model_runner = ModelTrainer(X_train=X_train_encoded,y_train=y_train['NPHIES_LABEL'],
                                X_test=X_test_encoded,y_test=y_test['NPHIES_LABEL'])

    model_runner.train_models()
    result_analysis = model_runner.evaluate_models()
    top_model_features = model_runner.get_xgboost_feature_importance()

    features_df = pd.DataFrame({"Column_name":X_test_encoded.columns,"Feature_importance":top_model_features}).sort_values(
        by=['Feature_importance'],ascending=False)
    model_features1 = find_service_importances(features_df)
    model_features = find_icd_importances(model_features1)
    return result_analysis['XGBoost'], model_features

def find_service_importances(model_features):
    combined_text_rows = model_features[model_features['Column_name'].str.contains('CombinedText')]
    combined_sum = sum(combined_text_rows['Feature_importance'])
    model_features = model_features[~model_features['Column_name'].str.contains('CombinedText')]
    model_features.loc[len(model_features)] = {'Column_name':'SERVICE_DESCRIPTION','Feature_importance':float(combined_sum)}
    return model_features

def find_icd_importances(model_features):
    combined_text_rows = model_features[model_features['Column_name'].str.contains('ICDText')]
    combined_sum = sum(combined_text_rows['Feature_importance'])
    model_features = model_features[~model_features['Column_name'].str.contains('ICDText')]
    model_features.loc[len(model_features)] = {'Column_name':'ICD_10','Feature_importance':float(combined_sum)}
    model_features.sort_values(by=['Feature_importance'],ascending=False,inplace=True)
    return model_features

def output_results_error(error_value):
    model_performance, model_features = analyze_error_code(error_value)
    df = pd.DataFrame(list(model_performance.items()))
    df.to_csv(f'data/Extras/2024_08_20/{error_value}_acc.csv',index=False)
    model_features.to_csv(f'data/Extras/2024_08_20/{error_value}_feats.csv',index=False)

In [10]:
error_value = 'MN-1-1'
model_performance, model_features = analyze_error_code(error_value)

E:\Projects\Claims_Rejection\src\data_local.py:275: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[column] = df[column].replace(column_encoding)
E:\Projects\Claims_Rejection\src\data_local.py:282: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['PatientAgeRange'] = df.PatientAgeRange.replace(age_encoding)
E:\Projects\Claims_Rejection\src\data_local.py:275: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy

[LightGBM] [Info] Number of positive: 17407, number of negative: 59599
[LightGBM] [Info] Total Bins 6554
[LightGBM] [Info] Number of data points in the train set: 77006, number of used features: 62
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.226047 -> initscore=-1.230766
[LightGBM] [Info] Start training from score -1.230766


LightGBM, Decision Tree, SGD and Neural Network are trained on dataset.


C:\Users\omar.mounir\AppData\Local\Temp\ipykernel_1648\1808676347.py:60: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.26108051068149507' has dtype incompatible with float32, please explicitly cast to a compatible dtype first.
  model_features.loc[len(model_features)] = {'Column_name':'SERVICE_DESCRIPTION','Feature_importance':float(combined_sum)}
C:\Users\omar.mounir\AppData\Local\Temp\ipykernel_1648\1808676347.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_features.loc[len(model_features)] = {'Column_name':'ICD_10','Feature_importance':float(combined_sum)}
C:\Users\omar.mounir\AppData\Local\Temp\ipykernel_1648\1808676347.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

In [11]:
print(f'Training for {error_value}, the model has: {model_performance}')

Training for MN-1-1, the model has: {'Accuracy': 0.89, 'Precision': 0.74, 'Recall': 0.57, 'F1 Score': 0.65}


In [12]:
print(model_features)

                 Column_name  Feature_importance
50       SERVICE_DESCRIPTION            0.261081
35                    ICD_10            0.076171
29            TREATMENT_TYPE            0.073752
30            PURCHASER_CODE            0.070696
15  PROVIDER_DEPARTMENT_CODE            0.049635
7             VAT_PERCENTAGE            0.042631
28           DEPARTMENT_TYPE            0.036681
5             NET_VAT_AMOUNT            0.035640
4         LINE_ITEM_DISCOUNT            0.034356
9                   DURATION            0.029308
11    UNIT_PRICE_STOCKED_UOM            0.021178
2    LINE_CLAIMED_AMOUNT_SAR            0.021047
13       DISCOUNT_PERCENTAGE            0.020230
12            UNIT_PRICE_NET            0.018411
3               CO_INSURANCE            0.014833
31               CONTRACT_NO            0.014754
16     DOCTOR_SPECIALTY_CODE            0.014365
1                        QTY            0.013922
24  TOTAL_CLAIMED_AMOUNT_SAR            0.013828
23                CL

In [5]:
error_value = 'CV-1-1'
model_performance, model_features = analyze_error_code(error_value)

E:\Projects\Claims_Rejection\src\data_local.py:275: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[column] = df[column].replace(column_encoding)
E:\Projects\Claims_Rejection\src\data_local.py:282: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['PatientAgeRange'] = df.PatientAgeRange.replace(age_encoding)
E:\Projects\Claims_Rejection\src\data_local.py:275: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy

[LightGBM] [Info] Number of positive: 10129, number of negative: 66877
[LightGBM] [Info] Total Bins 6554
[LightGBM] [Info] Number of data points in the train set: 77006, number of used features: 62
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.131535 -> initscore=-1.887453
[LightGBM] [Info] Start training from score -1.887453


LightGBM, Decision Tree, SGD and Neural Network are trained on dataset.


C:\Users\omar.mounir\AppData\Local\Temp\ipykernel_1648\1808676347.py:60: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.039837736170738935' has dtype incompatible with float32, please explicitly cast to a compatible dtype first.
  model_features.loc[len(model_features)] = {'Column_name':'SERVICE_DESCRIPTION','Feature_importance':float(combined_sum)}
C:\Users\omar.mounir\AppData\Local\Temp\ipykernel_1648\1808676347.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_features.loc[len(model_features)] = {'Column_name':'ICD_10','Feature_importance':float(combined_sum)}
C:\Users\omar.mounir\AppData\Local\Temp\ipykernel_1648\1808676347.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [6]:
print(f'Training for {error_value}, the model has: {model_performance}')

Training for CV-1-1, the model has: {'Accuracy': 0.9, 'Precision': 0.94, 'Recall': 0.63, 'F1 Score': 0.75}


In [7]:
model_features

,Column_name,Feature_importance
30,PURCHASER_CODE,0.292619
28,DEPARTMENT_TYPE,0.135752
29,TREATMENT_TYPE,0.121588
23,CLAIM_TYPE,0.057480
31,CONTRACT_NO,0.056405
20,PATIENT_NATIONALITY,0.049356
35,ICD_10,0.044195
50,SERVICE_DESCRIPTION,0.039838
16,DOCTOR_SPECIALTY_CODE,0.037601
25,TOTAL_DISCOUNT,0.016043


In [13]:
error_value = 'SE-1-6'
model_performance, model_features = analyze_error_code(error_value)

E:\Projects\Claims_Rejection\src\data_local.py:275: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[column] = df[column].replace(column_encoding)
E:\Projects\Claims_Rejection\src\data_local.py:282: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['PatientAgeRange'] = df.PatientAgeRange.replace(age_encoding)
E:\Projects\Claims_Rejection\src\data_local.py:275: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy

[LightGBM] [Info] Number of positive: 10031, number of negative: 66975
[LightGBM] [Info] Total Bins 6554
[LightGBM] [Info] Number of data points in the train set: 77006, number of used features: 62
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.130263 -> initscore=-1.898639
[LightGBM] [Info] Start training from score -1.898639


LightGBM, Decision Tree, SGD and Neural Network are trained on dataset.


E:\Projects\Claims_Rejection\src\model_train.py:59: RuntimeWarning: invalid value encountered in scalar divide
  mod_f1 = (2*mod_precision*mod_recall) / (mod_precision+mod_recall)
C:\Users\omar.mounir\AppData\Local\Temp\ipykernel_1648\1808676347.py:60: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.03420372254913673' has dtype incompatible with float32, please explicitly cast to a compatible dtype first.
  model_features.loc[len(model_features)] = {'Column_name':'SERVICE_DESCRIPTION','Feature_importance':float(combined_sum)}
C:\Users\omar.mounir\AppData\Local\Temp\ipykernel_1648\1808676347.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_features.loc[len(model_features)] = {'Column_name':'ICD_10','Feature_impo

In [14]:
print(f'Training for {error_value}, the model has: {model_performance}')

Training for SE-1-6, the model has: {'Accuracy': 0.99, 'Precision': 0.98, 'Recall': 0.97, 'F1 Score': 0.97}


In [16]:
model_features

,Column_name,Feature_importance
30,PURCHASER_CODE,0.463636
31,CONTRACT_NO,0.152568
1,QTY,0.043568
28,DEPARTMENT_TYPE,0.036525
50,SERVICE_DESCRIPTION,0.034204
26,TOTAL_DEDUCTIBLE,0.032257
35,ICD_10,0.028425
24,TOTAL_CLAIMED_AMOUNT_SAR,0.019450
2,LINE_CLAIMED_AMOUNT_SAR,0.019218
29,TREATMENT_TYPE,0.013412


In [11]:
error_value = 'SE-1-8'
model_performance, model_features = analyze_error_code(error_value)

E:\Projects\Claims_Rejection\src\data_local.py:269: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.df['PatientAgeRange'] = self.df.PatientAgeRange.replace(age_encoding)
E:\Projects\Claims_Rejection\src\data_local.py:269: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.df['PatientAgeRange'] = self.df.PatientAgeRange.replace(age_encoding)


[LightGBM] [Info] Number of positive: 1297, number of negative: 78392
[LightGBM] [Info] Total Bins 5683
[LightGBM] [Info] Number of data points in the train set: 79689, number of used features: 47
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.016276 -> initscore=-4.101668
[LightGBM] [Info] Start training from score -4.101668


LightGBM, Decision Tree, SGD and Neural Network are trained on dataset.


E:\Projects\Claims_Rejection\src\model_train.py:59: RuntimeWarning: invalid value encountered in scalar divide
  mod_f1 = (2*mod_precision*mod_recall) / (mod_precision+mod_recall)
E:\Projects\Claims_Rejection\src\model_train.py:59: RuntimeWarning: invalid value encountered in scalar divide
  mod_f1 = (2*mod_precision*mod_recall) / (mod_precision+mod_recall)
C:\Users\omar.mounir\AppData\Local\Temp\ipykernel_11324\3176289654.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_features.loc[len(model_features)] = {'Column_name':'SERVICE_DESCRIPTION','Feature_importance':float(combined_sum)}
C:\Users\omar.mounir\AppData\Local\Temp\ipykernel_11324\3176289654.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://

In [12]:
print(f'Training for {error_value}, the model has: {model_performance}') ## data is extremely imbalanced
## Predictive analytics couldn't be done

Training for SE-1-8, the model has: {'Accuracy': 0.98, 'Precision': 0.14, 'Recall': 0.04, 'F1 Score': 0.06}


In [13]:
model_features

,Column_name,Feature_importance
27,DEPARTMENT_TYPE,0.447325
29,PURCHASER_CODE,0.123270
33,SERVICE_DESCRIPTION,0.047021
19,PATIENT_NATIONALITY,0.046959
14,PROVIDER_DEPARTMENT_CODE,0.042484
24,TOTAL_DISCOUNT,0.040729
16,DOCTOR_CODE,0.036699
15,DOCTOR_SPECIALTY_CODE,0.029574
23,TOTAL_CLAIMED_AMOUNT_SAR,0.028882
17,PATIENT_AGE,0.026865


In [4]:
output_results_error('MN-1-1')
output_results_error('CV-1-1')
output_results_error('SE-1-6')
output_results_error('BE-1-7')
output_results_error('BE-1-4')
output_results_error('SE-1-8')

E:\Projects\Claims_Rejection\src\data_local.py:269: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.df['PatientAgeRange'] = self.df.PatientAgeRange.replace(age_encoding)
E:\Projects\Claims_Rejection\src\data_local.py:269: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.df['PatientAgeRange'] = self.df.PatientAgeRange.replace(age_encoding)


[LightGBM] [Info] Number of positive: 17007, number of negative: 62682
[LightGBM] [Info] Total Bins 6193
[LightGBM] [Info] Number of data points in the train set: 79689, number of used features: 49
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.213417 -> initscore=-1.304449
[LightGBM] [Info] Start training from score -1.304449


LightGBM, Decision Tree, SGD and Neural Network are trained on dataset.


C:\Users\omar.mounir\AppData\Local\Temp\ipykernel_9416\3176289654.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_features.loc[len(model_features)] = {'Column_name':'SERVICE_DESCRIPTION','Feature_importance':float(combined_sum)}
C:\Users\omar.mounir\AppData\Local\Temp\ipykernel_9416\3176289654.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_features.sort_values(by=['Feature_importance'],ascending=False,inplace=True)
E:\Projects\Claims_Rejection\src\data_local.py:269: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, 

[LightGBM] [Info] Number of positive: 12461, number of negative: 67228
[LightGBM] [Info] Total Bins 6193
[LightGBM] [Info] Number of data points in the train set: 79689, number of used features: 49
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.156370 -> initscore=-1.685486
[LightGBM] [Info] Start training from score -1.685486


LightGBM, Decision Tree, SGD and Neural Network are trained on dataset.


C:\Users\omar.mounir\AppData\Local\Temp\ipykernel_9416\3176289654.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_features.loc[len(model_features)] = {'Column_name':'SERVICE_DESCRIPTION','Feature_importance':float(combined_sum)}
C:\Users\omar.mounir\AppData\Local\Temp\ipykernel_9416\3176289654.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_features.sort_values(by=['Feature_importance'],ascending=False,inplace=True)
E:\Projects\Claims_Rejection\src\data_local.py:269: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, 

...

C:\Users\omar.mounir\AppData\Local\Temp\ipykernel_9416\3176289654.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_features.loc[len(model_features)] = {'Column_name':'SERVICE_DESCRIPTION','Feature_importance':float(combined_sum)}
C:\Users\omar.mounir\AppData\Local\Temp\ipykernel_9416\3176289654.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_features.sort_values(by=['Feature_importance'],ascending=False,inplace=True)
E:\Projects\Claims_Rejection\src\data_local.py:269: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, 

[LightGBM] [Info] Number of positive: 2595, number of negative: 77094
[LightGBM] [Info] Total Bins 6193
[LightGBM] [Info] Number of data points in the train set: 79689, number of used features: 49
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.032564 -> initscore=-3.391439
[LightGBM] [Info] Start training from score -3.391439


LightGBM, Decision Tree, SGD and Neural Network are trained on dataset.


E:\Projects\Claims_Rejection\src\model_train.py:59: RuntimeWarning: invalid value encountered in scalar divide
  mod_f1 = (2*mod_precision*mod_recall) / (mod_precision+mod_recall)
C:\Users\omar.mounir\AppData\Local\Temp\ipykernel_9416\3176289654.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_features.loc[len(model_features)] = {'Column_name':'SERVICE_DESCRIPTION','Feature_importance':float(combined_sum)}
C:\Users\omar.mounir\AppData\Local\Temp\ipykernel_9416\3176289654.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_features.sort_values(by=['Feature_importance'],ascending=False,inplace=True)
E:\P

[LightGBM] [Info] Number of positive: 1297, number of negative: 78392
[LightGBM] [Info] Total Bins 6193
[LightGBM] [Info] Number of data points in the train set: 79689, number of used features: 49
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.016276 -> initscore=-4.101668
[LightGBM] [Info] Start training from score -4.101668


LightGBM, Decision Tree, SGD and Neural Network are trained on dataset.


E:\Projects\Claims_Rejection\src\model_train.py:59: RuntimeWarning: invalid value encountered in scalar divide
  mod_f1 = (2*mod_precision*mod_recall) / (mod_precision+mod_recall)
E:\Projects\Claims_Rejection\venv\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
E:\Projects\Claims_Rejection\src\model_train.py:59: RuntimeWarning: invalid value encountered in scalar divide
  mod_f1 = (2*mod_precision*mod_recall) / (mod_precision+mod_recall)
C:\Users\omar.mounir\AppData\Local\Temp\ipykernel_9416\3176289654.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_feature

In [5]:
output_results_error('BE-1-1')
output_results_error('BE-1-3')
output_results_error('BE-1-6')

E:\Projects\Claims_Rejection\src\data_local.py:269: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.df['PatientAgeRange'] = self.df.PatientAgeRange.replace(age_encoding)
E:\Projects\Claims_Rejection\src\data_local.py:269: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.df['PatientAgeRange'] = self.df.PatientAgeRange.replace(age_encoding)


[LightGBM] [Info] Number of positive: 3114, number of negative: 76575
[LightGBM] [Info] Total Bins 6193
[LightGBM] [Info] Number of data points in the train set: 79689, number of used features: 49
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.039077 -> initscore=-3.202363
[LightGBM] [Info] Start training from score -3.202363


LightGBM, Decision Tree, SGD and Neural Network are trained on dataset.


C:\Users\omar.mounir\AppData\Local\Temp\ipykernel_9416\3176289654.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_features.loc[len(model_features)] = {'Column_name':'SERVICE_DESCRIPTION','Feature_importance':float(combined_sum)}
C:\Users\omar.mounir\AppData\Local\Temp\ipykernel_9416\3176289654.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_features.sort_values(by=['Feature_importance'],ascending=False,inplace=True)
E:\Projects\Claims_Rejection\src\data_local.py:269: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, 

[LightGBM] [Info] Number of positive: 4125, number of negative: 75564
[LightGBM] [Info] Total Bins 6193
[LightGBM] [Info] Number of data points in the train set: 79689, number of used features: 49
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.051764 -> initscore=-2.907914
[LightGBM] [Info] Start training from score -2.907914


LightGBM, Decision Tree, SGD and Neural Network are trained on dataset.


E:\Projects\Claims_Rejection\src\model_train.py:59: RuntimeWarning: invalid value encountered in scalar divide
  mod_f1 = (2*mod_precision*mod_recall) / (mod_precision+mod_recall)
E:\Projects\Claims_Rejection\src\model_train.py:59: RuntimeWarning: invalid value encountered in scalar divide
  mod_f1 = (2*mod_precision*mod_recall) / (mod_precision+mod_recall)
C:\Users\omar.mounir\AppData\Local\Temp\ipykernel_9416\3176289654.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_features.loc[len(model_features)] = {'Column_name':'SERVICE_DESCRIPTION','Feature_importance':float(combined_sum)}
C:\Users\omar.mounir\AppData\Local\Temp\ipykernel_9416\3176289654.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pa

[LightGBM] [Info] Number of positive: 4068, number of negative: 75621
[LightGBM] [Info] Total Bins 6193
[LightGBM] [Info] Number of data points in the train set: 79689, number of used features: 49
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.051048 -> initscore=-2.922583
[LightGBM] [Info] Start training from score -2.922583


LightGBM, Decision Tree, SGD and Neural Network are trained on dataset.


E:\Projects\Claims_Rejection\venv\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
E:\Projects\Claims_Rejection\src\model_train.py:59: RuntimeWarning: invalid value encountered in scalar divide
  mod_f1 = (2*mod_precision*mod_recall) / (mod_precision+mod_recall)
C:\Users\omar.mounir\AppData\Local\Temp\ipykernel_9416\3176289654.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_features.loc[len(model_features)] = {'Column_name':'SERVICE_DESCRIPTION','Feature_importance':float(combined_sum)}
C:\Users\omar.mounir\AppData\Local\Temp\ipykernel_9416\3176289654.py:56: